In [21]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
import random
import seaborn as sns
#import matplotlib as mpl
from sklearn.model_selection import train_test_split

tfk = tf.keras
tfkl = tf.keras.layers

In [22]:
# Random seed for reproducibility
seed = 1

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [23]:
# Utility function to create folders and callbacks for training
from datetime import datetime

exp_dir = ''

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%Y-%m-%d_%H:%M')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tfk.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp'),
                                                save_weights_only=False,
                                                save_best_only=False)
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                               profile_batch=0,
                                               histogram_freq=1)
  callbacks.append(tb_callback)

In [24]:
#!unzip ANN_HW1.zip
#import zipfile
#with zipfile.ZipFile("dataset.zip", 'r') as z:
#    z.extractall()

In [25]:
# Set the path to the dataset 

dataset_dir = '../input/homework1split/'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')

In [26]:
# Plot example images from dataset
'''
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

num_row = len(labels)//2
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
for i in range(len(labels)):
    if i < len(labels):
        class_imgs = next(os.walk('{}/{}/'.format(training_dir, labels[i])))[2]
        class_img = class_imgs[0]
        img = Image.open('{}/{}/{}'.format(training_dir, labels[i], class_img))
        ax = axes[i//num_col, i%num_col]
        ax.imshow(np.array(img))
        ax.set_title('{}'.format(labels[i]))
plt.tight_layout()
plt.show()
'''

"\nlabels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']\n\nnum_row = len(labels)//2\nnum_col = len(labels)//num_row\nfig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))\nfor i in range(len(labels)):\n    if i < len(labels):\n        class_imgs = next(os.walk('{}/{}/'.format(training_dir, labels[i])))[2]\n        class_img = class_imgs[0]\n        img = Image.open('{}/{}/{}'.format(training_dir, labels[i], class_img))\n        ax = axes[i//num_col, i%num_col]\n        ax.imshow(np.array(img))\n        ax.set_title('{}'.format(labels[i]))\nplt.tight_layout()\nplt.show()\n"

In [27]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input) 
# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method

train_dataset = train_gen.flow_from_directory(directory=training_dir,
                                                target_size=(256,256),
                                                color_mode='rgb',
                                                classes=None,
                                                batch_size=8,
                                                shuffle=True,
                                                class_mode='categorical',
                                                seed=seed)


valid_gen = ImageDataGenerator(preprocessing_function=preprocess_input)   

validation_dataset = valid_gen.flow_from_directory(directory=validation_dir,
                                                target_size=(256,256),
                                                color_mode='rgb',
                                                classes=None,
                                                batch_size=8,
                                                shuffle=True,
                                                class_mode='categorical',
                                                seed=seed)

Found 14176 images belonging to 14 classes.
Found 3552 images belonging to 14 classes.


In [28]:
input_shape = (256, 256, 3)
epochs = 150
model_name = 'inception'

In [29]:
def build_model(input_shape):
#create model

    inception = tfk.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    inception.trainable = False

    model = tfk.Sequential()
    model.add(inception)
    model.add(tfk.layers.GlobalAveragePooling2D())
    model.add(tfk.layers.Dense(512, activation='relu'))
    model.add(tfk.layers.Dropout(0.5))
    model.add(tfk.layers.Dense(256, activation='relu'))
    model.add(tfk.layers.Dropout(0.5))
    model.add(tfk.layers.Dense(14, activation='relu'))
    model.add(tfk.layers.Softmax())

    model.compile(optimizer=tfk.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [30]:
model = build_model(input_shape)
model.summary()
tfk.utils.plot_model(model)

53805056/87910968 [=================>............] - ETA: 5s

KeyboardInterrupt: 

In [ ]:
callbacks = create_folders_and_callbacks(model_name=model_name)

# Train the model
history = model.fit(
    x = train_dataset,
    epochs = epochs,
    batch_size = 256,
    validation_data = validation_dataset,
    callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)]
).history

In [ ]:
# Save the model
model.save(os.path.join(exp_dir, model_name))

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()